In [1]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
#from src import filter_assets
from src import plot
from src import portfolio
from src import capm
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
import datetime
from dateutil import rrule
import yfinance as yf
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='pandas')

Using matplotlib backend: <object object at 0x10fa9b8f0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Using matplotlib backend: MacOSX
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
x = pd.Timestamp(np.datetime64('2000-01-01'))
x1 = pd.Timestamp(np.datetime64('2022-01-01'))

In [3]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1400
max_esg_score = 2000
df = pd.read_excel('../data/ESG_DATA_S&P500.xlsx')
dates = [x,x1]

start_year = dates[0]
end_year = dates[1]

Bounds1 = Bounds(-1,2) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01
covariance_window = 5 #has to be in years



esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type,start_year,end_year, Wanted_return, maximum_risk,'yearly') #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters) #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.
sp500 = data.data_for_beta(prices)
pct_returns_sp500 = data.pct_returns_from_prices(sp500)
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio,'SPY')
capm_expected_return = capm.capm_calc(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_of_portfolio)

strategy1 = {'df': df,
 'weights': weights,
 'min_esg_score': min_esg_score,
 'max_esg_score': max_esg_score,
 'Bounds1': Bounds1,
 'sharpe_type': sharpe_type,
 'x': x,
 'x1': x1,
 'Wanted_return': Wanted_return,
 'maximum_risk': maximum_risk,
 'rebalancing_freq': 'monthly',
 'risk_free_rate': risk_free_rate}




Min. Risk = 27.534% => Return: 15.192%  Sharpe Ratio = 0.55
Max. Sharpe Ratio = 0.66 => Return: 21.71%  Risk: 32.914%
Excpected return on investment is 121.4320990581248%


In [4]:
# import packages
from datetime import date
from dateutil.relativedelta import relativedelta
import warnings

# Suppress the FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning) #We get a weird warning, a bug with pandas according to their github
#the dates are a little fnky
def backtesting(strategy,monthly_or_yearly_rebalancing,rebalancing_freq,start_date,end_date,covariance_window,market_name):
    
    #start = 2015
   # end = 2022

    covariance_window_time_delta = relativedelta(years=covariance_window) #The time delta for the covariance window
    #print([start_date-covariance_window_time_delta,end_date])
    esg_data = data.esg_score_weight(strategy['df'],strategy['weights'],strategy['min_esg_score'],strategy['max_esg_score'])
    full_data = data.stock_monthly_close(esg_data,[pd.Timestamp(start_date-covariance_window_time_delta),pd.Timestamp(end_date+relativedelta(years=rebalancing_freq,months=1))])#Making sure we download enough data
    prices,esgdata = data.seperate_full_data(full_data)
    pct_returns = data.pct_returns_from_prices(prices)
    stock_data_download = yf.download(market_name, start=dates[0]-covariance_window_time_delta, end=end_date+relativedelta(years=rebalancing_freq+1), interval='1mo', progress=False)
    stock_data_download = stock_data_download[['Close']].rename(columns={'Close': market_name})
    #print(prices)
    #print(stock_data_download)

    stock_data = pd.concat([prices,stock_data_download], axis = 1)
    #print(stock_data)
    pct = data.pct_returns_from_prices(stock_data)[start_date-covariance_window_time_delta:end_date+relativedelta(years=rebalancing_freq+1)]
    #print(pct)
    pct.index =pct.index.tz_localize(None)
    #print(pct)
    #print(prices)
    #print(prices)
    #print(stock_data_download.loc[start_date+delta][0]-stock_data_download.loc[start_date][0]/stock_data_download.loc[start_date][0])
    #print(prices[start_date-covariance_window_time_delta:end_date])

   


    
    listparameters = []
    list_of_port_weights =[]
    list_of_port_esg_scores = []
    list_of_port_allocations = []
    list_of_cmle_returns = []
    list_of_portfolio_actual_returns = []
    sp500_list = []
    list_of_pct_returns_sp500 = []
    betas_of_portfolios = []
    capm_for_portfolio = []
    list_of_cml_allocations = []
    list_of_return_dates = []
    i = 0

    if monthly_or_yearly_rebalancing == 'yearly':
        delta = relativedelta(years=rebalancing_freq)
        
        while (start_date <= end_date):
            #print(start_date-covariance_window_time_delta,start_date)
            
            



            

            listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date-covariance_window_time_delta,#The start date for portfolio optimization will be the start date minus the covariance window
                            start_date,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy['rebalancing_freq'])) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
            list_of_cml_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            betas_of_portfolios.append(capm.calculate_portfolio_beta(pct,prices[start_date-covariance_window_time_delta:start_date],list_of_port_weights[i],market_name))            
            capm_for_portfolio.append(capm.capm_calc(pct[market_name][start_date-covariance_window_time_delta:start_date].tz_localize(None).mean(),risk_free_rate,betas_of_portfolios[i]))

            list_of_portfolio_actual_returns.append(((portfolio.portfolio_return(prices.loc[start_date+delta],list_of_port_weights[i].T)[0])
                                                     *(1-list_of_cml_allocations[i][0])-(portfolio.portfolio_return(prices.loc[start_date],list_of_port_weights[i].T)[0])
                                                     *(1-list_of_cml_allocations[i][0]))/(portfolio.portfolio_return(prices.loc[start_date],list_of_port_weights[i].T)[0])
                                                     *(1-list_of_cml_allocations[i][0])+list_of_cml_allocations[i][0]*risk_free_rate)#list of how large a percentage you would have made using your investment strategy
            #List of portfolios actual returns skal laves om så man tager højde cml allokeringen
            list_of_pct_returns_sp500.append((stock_data_download.loc[start_date+delta][0]-stock_data_download.loc[start_date][0])/stock_data_download.loc[start_date][0])#List of how large a percentage you would have made investing everything in the market

            list_of_return_dates.append((start_date+delta).strftime('%Y/%m/%d'))
            #print(list_of_portfolio_actual_returns)
           # print(list_of_return_dates)




            start_date += delta
            
            
            
            i += 1

        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,betas_of_portfolios,list_of_cmle_returns,list_of_portfolio_actual_returns,list_of_pct_returns_sp500,list_of_return_dates)

    elif monthly_or_yearly_rebalancing == 'monthly':
        delta = delta = np.timedelta64(rebalancing_freq,'M')
        while (start_date <= end_date):
            
            
            



            

            listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date-covariance_window_time_delta,#The start date for portfolio optimization will be the start date minus the covariance window
                            start_date,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy['rebalancing_freq'])) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
            list_of_cml_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            betas_of_portfolios.append(capm.calculate_portfolio_beta(pct,prices[start_date-covariance_window_time_delta:start_date],list_of_port_weights[i],market_name))            
            capm_for_portfolio.append(capm.capm_calc(pct[market_name][start_date-covariance_window_time_delta:start_date].tz_localize(None).mean(),risk_free_rate,betas_of_portfolios[i]))

            list_of_portfolio_actual_returns.append(((portfolio.portfolio_return(prices.loc[start_date+delta],list_of_port_weights[i].T)[0])
                                                     *(1-list_of_cml_allocations[i][0])-(portfolio.portfolio_return(prices.loc[start_date],list_of_port_weights[i].T)[0])
                                                     *(1-list_of_cml_allocations[i][0]))/(portfolio.portfolio_return(prices.loc[start_date],list_of_port_weights[i].T)[0])
                                                     *(1-list_of_cml_allocations[i][0])+list_of_cml_allocations[i][0]*risk_free_rate)#list of how large a percentage you would have made using your investment strategy
            #List of portfolios actual returns skal laves om så man tager højde cml allokeringen
            list_of_pct_returns_sp500.append((stock_data_download.loc[start_date+delta][0]-stock_data_download.loc[start_date][0])/stock_data_download.loc[start_date][0])#List of how large a percentage you would have made investing everything in the market

            list_of_return_dates.append((start_date+delta).strftime('%Y/%m/%d'))
            print(list_of_portfolio_actual_returns)
            print(list_of_pct_returns_sp500)




            start_date += delta
            
            
            
            i += 1

        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,betas_of_portfolios,list_of_return_dates,list_of_cmle_returns,list_of_pct_returns_sp500)

        
    
    else:
        print('You can only call this function with monthly or yearly')
        return(False)
    

data5 = backtesting(strategy1,'yearly',1,datetime.datetime(2015,1,1),datetime.datetime(2022,1,1),10,'SPY')
#backtesting(strategy1,'yearly',1,np.datetime64('2015-01-01'),np.datetime64('2022-01-01'),10) #US dates YYYY-MM-DD

#This makes each portfolio using strategy 1 rebalancing every 6 months from 2015 until 2022, using a covariance window of ten years

#This makes each portfolio for 


Min. Risk = 6.540% => Return: 0.888%  Sharpe Ratio = 0.14
Max. Sharpe Ratio = 0.21 => Return: 2.10%  Risk: 10.057%
Excpected return on investment is 114.77570342293917%
Min. Risk = 6.876% => Return: 1.230%  Sharpe Ratio = 0.18
Max. Sharpe Ratio = 0.25 => Return: 2.31%  Risk: 9.429%
Excpected return on investment is 113.8703143634183%
Min. Risk = 6.818% => Return: 1.390%  Sharpe Ratio = 0.20
Max. Sharpe Ratio = 0.29 => Return: 2.84%  Risk: 9.731%
Excpected return on investment is 116.3900276073643%
Min. Risk = 6.553% => Return: 1.437%  Sharpe Ratio = 0.22
Max. Sharpe Ratio = 0.29 => Return: 2.52%  Risk: 8.683%
Excpected return on investment is 115.77048025078622%
Min. Risk = 5.986% => Return: 2.059%  Sharpe Ratio = 0.34
Max. Sharpe Ratio = 0.40 => Return: 2.76%  Risk: 6.927%
Excpected return on investment is 111.48179429401755%
Min. Risk = 5.765% => Return: 1.874%  Sharpe Ratio = 0.33
Max. Sharpe Ratio = 0.35 => Return: 2.14%  Risk: 6.163%
Excpected return on investment is 108.466922131

In [5]:
#len(data5)
#print(data5[7],data5[6],data5[5])

plot.compare_returns1(data5[7],data5[5],data5[6])